In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Smither for varock west anvils

mm_floor = [94,87,54]
mm_ew_road = [124,122,115]
mm_s_road = [131,123,89]

#final = load_image('iron_plate.png')
final = load_image('steel_plate.png')
raw_req = 5

#final = load_image('iron_arrowheads.png')
#raw_req = 1

#final = load_image('iron_bolts.png')
#raw_req = 1

#raw = load_image('iron_bar.png')
raw = load_image('steel_bar.png')
bmp_tol = 0.2

target()
total_trips = 0
logins = 0
last_smith = mark_time()
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    if mark_time() - last_smith > 10*60:
        raise RuntimeError('Sorry folks, got lost somehow and now we\'re done.')
    minimap = get_minimap()
    masked = np.full_like(minimap,255)
    masked[minimap_mask] = minimap[minimap_mask]
    minimap = masked
    inv_full = count_inv() == 28
    a = find_colors(mm_ew_road,minimap,tol=(0.02,0.03,0.03),mode='hsl') #mm_ew_road
    b = find_colors(mm_s_road,minimap,tol=(0.02,0.03,0.03),mode='hsl') #mm_s_road
    c = find_colors(mm_floor,minimap,tol=(0.02,0.03,0.03),mode='hsl') #mm_floor
    map_dark = find_colors([0,0,0],minimap,0.001)

    anvil_bldg = filter_near(filter_near(c,a,10),b,10)
    npcs = find_colors([255,255,0],minimap,tol=0.05,mode='hsl')

    bankers = filter_far(filter_near(npcs,map_dark,30),b,30)
    
    inventory = get_inventory()
    raw_points = find_bitmap(raw,inventory,mode='hsl',tol=bmp_tol)
    
    if inv_full or len(raw_points) > 1.5*raw_req: #go smith
        if len(anvil_bldg) == 0:
            print('can\'t find anvil building!')
            click_mouse(mmxc,mmyc+20)
            continue
        np.random.shuffle(anvil_bldg)
        click_mouse(*(anvil_bldg[0]+[mmxs+5,mmys+5]))
        sleep(1.0)
        flag_wait()
        if len(raw_points):
            np.random.shuffle(raw_points)
            click_mouse(*(raw_points[0]+[ivxs,ivys]))
            mainscreen = get_mainscreen()
            a = find_colors([50,50,50],mainscreen,tol=0.05,mode='hsl')
            b = find_colors([100,100,100],mainscreen,tol=0.05,mode='hsl')
            points = filter_near(b,a,10)
            np.random.shuffle(points)
            for point in points[-5:]:
                move_mouse(*(point+[msxs,msys]))
                sleep(0.5)
                uptext,mask = uptext_mask(get_uptext())
                txt = image_to_string(mask)
                print(txt)
                if 'ﬂmnl' in txt or 'nml' in txt or 'nwl' in txt or 'nyml' in txt:
                    click_mouse(*(point+[msxs,msys]))
                    flag_wait()
                    mainscreen = get_mainscreen()
                    smith_pos = find_bitmap(final,mainscreen,tol=bmp_tol,mode='hsl')
                    if len(smith_pos) ==0:
                        break
                    np.random.shuffle(smith_pos)
                    click_mouse(*(smith_pos[0]+[msxs,msys]),left=False)
                    sleep(2.0)
                    print('make 10')
                    click_mouse(*(smith_pos[0]+[msxs,msys+58]))
                    last_smith = mark_time()
                    polish_minimap(click=False)
                    run_on()
                    raw_num = len(raw_points)
                    i = 0
                    while True:
                        i = i+1
                        inventory = get_inventory()
                        new_raw_num = len(find_bitmap(raw,inventory,mode='hsl',tol=bmp_tol))
                        print('remaining:',new_raw_num)
                        if new_raw_num != raw_num:
                            raw_num = new_raw_num
                            i = 0
                        if i > 5:
                            break
                        if new_raw_num < raw_req:
                            break
                        sleep(0.5)
                    break
        else:
            print('no raw material found!')
    else: #go bank
        #open bank    
        mainscreen = get_mainscreen()
        if len(find_bitmap(bank_window,mainscreen,tol=0.02)) == 0:
            print('going to bank')
            if len(bankers) == 0:
                continue
            np.random.shuffle(bankers)
            click_mouse(*(bankers[0]+[mmxs,mmys]))
            flag_wait()
            
            mainscreen = get_mainscreen()
            a = find_colors([60,60,60],mainscreen,tol=0.05,mode='hsl')
            b = find_colors([55,45,87],mainscreen,tol=0.05,mode='hsl')
            points = filter_near(a,b,50) 
            print(len(a),len(b),len(points))
            np.random.shuffle(points)
            for point in points[-5:]:
                click_mouse(*point,left=False)
                sleep(0.5)
                use = find_bitmap(use_booth,get_mainscreen())
                if len(use) > 0:
                    click_mouse(*(use[0]+[10,10]))
                    sleep(1.0)
                    flag_wait()
                    print('bank opened')
                    sleep(1.0)
                    break
                move_mouse(*(point+[0,-25]))
            continue
        else:
            inventory = get_inventory()
            final_points = find_bitmap(final,inventory,mode='hsl',tol=bmp_tol)
            if len(final_points):
                print('depositing product')
                np.random.shuffle(final_points)
                click_mouse(*final_points[0]+[ivxs,ivys],left=False)
                sleep(0.5)
                client = get_client()
                found = find_bitmap(store_all,client)
                if len(found):
                    click_mouse(*found[0]+[10,10])
                    sleep(1.0)
                    continue
            mainscreen = get_mainscreen()        
            raw_points = find_bitmap(raw,mainscreen,mode='hsl',tol=bmp_tol)
            if len(raw_points):
                print('withdrawing raw')
                np.random.shuffle(raw_points)
                click_mouse(*(raw_points[0]+[msxs,msys]),left=False)
                sleep(1.0)
                click_mouse(*(raw_points[0]+[msxs,msys]+[0,87]))
                sleep(1.0)
                send_keys('28\n')
                sleep(1.0)